### Google
- 기능 : geocode(keyword), reverse geocode
- 대한민국에는 google map platform에서 geocode 밖에 사용할만게 없다. direction(route)도 미지원이여서... 
- google map platform <https://mapsplatform.google.com/>
<br></br>
- 가격 정책 : 처음 사용자는 300달러의 무료 크레딧을 주고, 이후 카드 등록을 해놓으면 매달 200달러의 무료 크레딧으로 사용자가 원하는 기능을 한도내에서 사용하면,따로 과금은 되지 않는다(geocode : 1회 0.005달러, 1000회 5달러).

```Python 예제```

```- geocode```

In [1]:
import googlemaps 

def google_geocode(place):
    gmaps = googlemaps.Client(key='your-api-key')
    geocode_result = gmaps.geocode(place, language='ko')
    
    return geocode_result[0]['geometry']['location']

In [12]:
place = '대전광역시 대덕구 한남로 70'
location = google_geocode(place)

print(location)

{'lat': 36.3544294, 'lng': 127.4227297}


```- reverse geocode```

In [2]:
import googlemaps 

### reverse geocode 주소를 넣으면, 후보지를 뽑아줌 -> 후보지는 세부주소,구,군,시 규모 별 규모 별로 반환해줌
# 반환값은 geocode랑 동일
# 첫 번째 값 제일 주소와 동일한 주소 -> 해당 코드에서는 첫 번째 값 주소 이름만 반환
def google_reverse_geocode(lat,lon):
    gmaps = googlemaps.Client(key='your-api-key')
    reverse_geocode_result = gmaps.reverse_geocode((lat, lon), language='ko')
    
    return reverse_geocode_result[0]['formatted_address']

In [4]:
place_name = google_reverse_geocode(36.3544548, 127.4211446)
# input : 위도, 경도 
print(place_name)

대한민국 대전광역시 대덕구 오정동 한남로 70


---
### Kakao
- 기능 : geocode , reverse geocode, keyword, 행정동코드 (direction은 따로 제공하고 있지 않음)
- kakao developer https://developers.kakao.com/
<br></br>
- 가격정책 : Kakao Developers 로컬 10만/일, 300만/월 무료 (이용량 제한- 과금 시스템 따로 없음)

```Python```

```- geocode```

In [3]:
import requests
import json

### 주소 좌표 반환 
def kakao_geocode(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    headers = {"Authorization": "KakaoAK your-api-key"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    lon = api_json['documents'][0]['x'] # lon
    lat = api_json['documents'][0]['y'] # lat
    return [float(lat), float(lon)]

In [10]:
place = '대전광역시 대덕구 한남로 70'
location = kakao_geocode(place)

print(location)

[36.352799521652, 127.423309425352]


```- reverse geocode```

In [4]:
import requests
import json

### 좌표가 위치한 지번 주소, 도로명 주소
# 반환값 : [지번 주소, 도로명 주소]
def kakao_reverse_geocode(lat, lon):
    url = f'https://dapi.kakao.com/v2/local/geo/coord2address.json?x={lon}&y={lat}' 
    headers = {"Authorization": "KakaoAK your-api-key"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    address = [api_json['documents'][0]['address']['address_name'], api_json['documents'][0]['road_address']['address_name']]
    return address

In [8]:
place_name = kakao_reverse_geocode(36.3544548, 127.4211446)

print(place_name)

['대전 대덕구 오정동 133', '대전광역시 대덕구 한남로 70']


```- region code```

In [5]:
import requests
import json

### 좌표가 위치한 법정동,행정동 코드
# 반환값 : [법정동 코드, 행정동 코드]
def kakao_regioncode(lat, lon):
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={lon}&y={lat}'
    headers = {"Authorization": "KakaoAK your-api-key"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    code = [api_json['documents'][0]['code'], api_json['documents'][1]['code']] # 법정동, 행정동
    return code

In [10]:
location_region_code = kakao_regioncode(36.3544548, 127.4211446)

print(location_region_code)

['3023010100', '3023051000']


```
- keyword
    - 중심점을 기준으로 radius(meter) 범위의 keyword의 위치 정보를 반환 
    - radius가 아닌 격자 형식으로도 가능하다(자세한 내용은 홈페이지 참고).
```

In [14]:
import requests
import json

def kakao_keyword(keywords, location, radius_meter=20000, page_num=45):
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={keywords}&x={location["lon"]}&y={location["lat"]}&radius={radius_meter}&page={page_num}'
    headers = {"Authorization": "KakaoAK your-api-key"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    return api_json

In [16]:
keywords = '맛집'
location = {'lat':36.352799521652, 'lon':127.423309425352}

keyword_result = kakao_keyword(keywords, location)
print(keyword_result)

{'documents': [{'address_name': '대전 대덕구 중리동 411-2', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 육류,고기', 'distance': '837', 'id': '8407374', 'phone': '042-672-4288', 'place_name': '일미구이 중리점', 'place_url': 'http://place.map.kakao.com/8407374', 'road_address_name': '대전 대덕구 중리로 14', 'x': '127.425478140755', 'y': '36.3601409714937'}, {'address_name': '대전 대덕구 오정동 172-5', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 중식', 'distance': '235', 'id': '1482421525', 'phone': '042-622-7941', 'place_name': '야미마라탕 대전점', 'place_url': 'http://place.map.kakao.com/1482421525', 'road_address_name': '대전 대덕구 한남로 61', 'x': '127.421397385121', 'y': '36.3513383585919'}, {'address_name': '대전 대덕구 오정동 483-6', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 육류,고기 > 곱창,막창', 'distance': '924', 'id': '8278157', 'phone': '042-673-8266', 'place_name': '구들마루 오정점', 'place_url': 'http://place.map.kaka

---
### Naver 
- 기능 : geocode, reverse geocode, direction(5, 15) (keyword 검색 미지원)
- direction5, direction15 차이는 경유지 추가 가능 갯수 및 자세한 건 홈페이지 참고
- direction 기능은 호출 시간 기준이기 때문에 주의!!
- naver cloud platform https://www.ncloud.com/?language=ko-KR
<br></br>
- 가격 정책
    - geocode : 무료 3,000,000건/일(월) (그 이상 사용시 0.4원/건 (1,001~10,000,000건/일) | 0.3원/건 (10,000,000건 초과/일))
    - reverse geocode : 무료 3,000,000건/일(월) (그 이상 사용시 0.4원/건 (1,001~10,000,000건/일) | 0.3원/건 (10,000,000건 초과/일))
    - direction5 : 무료 6,000건/일 | 60,000건/월 (그 이상 사용시 5원/건)
    - direction15 : 무료 6,000건/일 | 60,000건/월(그 이상 사용시 20원/건)
    - 참고) 카드를 미리 등록하고 이용(무료 서비스 포함)

```Python```

```- geocode```

In [7]:
import json 
import requests

def naver_geocode(place):
    client_id = 'your-client-id'
    client_secret = 'your-client-scret-key' 
    query = place
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={query}'
    headers = {'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret}
    res= requests.get(url, headers=headers)
    data = res.json()
    try:
        lat = data['addresses'][0]['y']
        lon = data['addresses'][0]['x']
        return [float(lat), float(lon)]
    except:
        return []

In [14]:
place = '대전광역시 대덕구 한남로 70'
location = naver_geocode(place)

print(location)

[36.3541173, 127.4217991]


```
- reverse geocode
    - orders : legalcode 법정동 코드 | admcode 행정동 코드 | addr 지번 주소 | roadaddr 도로명 주소 
    - 지번 주소, 도로명 주소는 전체 주소를 반환 해주지 않아서.. 쓸모 있는지는 모르겠음
    - 대신 도로명 주소 호출은 해당 좌표 시군구 중심좌표,우편번호 얻을 때는 쓸만 할 듯
```

In [8]:
import json 
import requests

# 행정동, 법정동
def naver_region_code(lat, lon):
    client_id = 'your-client-id'
    client_secret = 'your-client-scret-key' 
    url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={lon},{lat}&output=json&orders=admcode,legalcode'
    headers = {'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret}
    res= requests.get(url, headers=headers)
    data = res.json()
    return [data['results'][0]['code']['id'], data['results'][1]['code']['id']] 

# 우편번호
def naver_zip_code(lat, lon):
    client_id = 'your-client-id'
    client_secret = 'your-client-scret-key' 
    url = f'https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={lon},{lat}&output=json&orders=roadaddr'
    headers = {'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret}
    res= requests.get(url, headers=headers)
    data = res.json()
    return data['results'][0]['land']['addition1']['value']

In [16]:
region_code = naver_region_code(36.3541173, 127.4217991)
zip_code = naver_zip_code(36.3541173, 127.4217991)

print(region_code)
print(zip_code)

['3023051000', '3023010100']
34430


```
- direction5
    - duration : 마이크로초 (마이크로초 1000 == 1초)
    - distance : 미터 
    - path : 루트(경로 좌표 데이터)
    - 나머지 생략...
```

In [1]:
import json
import requests

# O :[lat, lon] | D :[lat,lon]
def naver_direction(O, D): 
    client_id = 'your-client-id'
    client_secret = 'your-client-scret-key' 
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={O[1]},{O[0]}&goal={D[1]},{D[0]}"
    headers = {'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret}
    res= requests.get(url, headers=headers)
    data = res.json()
    
    if not('route' in data.keys()):
        return {}
    
    distance = data['route']['traoptimal'][0]['summary']['distance']
    duration = data['route']['traoptimal'][0]['summary']['duration']
    route = data['route']['traoptimal'][0]['path']
    return {'distance':distance, 'duration':duration, 'route':route}

In [18]:
O = [36.352799521652, 127.423309425352] # 한남대학교
D = [36.3512760051536, 127.437518299333] # 대전복합터미널

data = naver_direction(O,D)

### Tmap
- 기능 : geocode, reverse geocode, POI(keyword 검색),direction(현재, 과거, 보행자, 화물차)
<br></br>
- 가격정책 (정액제, 종량제)
    - geocode : 무료 20,000건/일(종량제 0.55원/건)
    - geofencing : 
    - reverse geocode : 무료 20,000건/일(종량제 0.55원/건)
    - POI 검색 : 무료 20,000건/일(종량제 1.1원/건)
    - 경로 안내(direction) : 무료 1,000건/일 (종량제 11원/건)
        - 자동차 경로안내
        - 보행자 경로안내
        - 타임머신 경로안내
        - 화물차 경로안내

```- reverse geocode```

In [10]:
import requests
import json 

# address [행정동주소, 법정동주소, 도로명주소], region_code [행정동코드, 법정동코드]
def tmap_reverse_geocode(lat, lon):
    url = f"https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1&lat={lat}&lon={lon}&coordType=WGS84GEO&addressType=A10&newAddressExtend=Y"

    headers = {
        "Accept": "application/json",
        "appKey": "your-appkey"
    }

    response = requests.get(url, headers=headers)
    res = json.loads(response.text)
    address = res['addressInfo']['fullAddress'].split(',')
    region_code = [res['addressInfo']['adminDongCode'], res['addressInfo']['legalDongCode']]
    return address, region_code

In [167]:
result = tmap_reverse_geocode(36.352799521652, 127.423309425352)

print(result)

(['대전광역시 대덕구 오정동', '대전광역시 대덕구 오정동 133', '대전광역시 대덕구 한남로 70 한남대학교'], ['3023051000', '3023010100'])


```- direction```

In [2]:
import requests
import json

def tmap_direction_result_process(result):
    routes  = []

    for idx, i in enumerate(result['features']):
        if idx == 0:
            duration = i['properties']['totalTime']
            distance = i['properties']['totalDistance']
        if i['geometry']['type']  == 'LineString':
            routes.extend(i['geometry']['coordinates'])
    return duration, distance, routes

def tmap_past_direction(O, D, yymmdd, hour, min):
    O_lat, O_lon = O[0], O[1]    
    D_lat, D_lon = D[0], D[1]
    
    url = "https://apis.openapi.sk.com/tmap/routes/prediction?version=1&resCoordType=WGS84GEO&reqCoordType=WGS84GEO&sort=index&callback=function"

    payload = {"routesInfo": {
            "departure": {
                "name": "test1",
                "lon": f"{O_lon}",
                "lat": f"{O_lat}",
                "depSearchFlag": "03"
            },
            "destination": {
                "name": "test2",
                "lon": f"{D_lon}",
                "lat": f"{D_lat}",
                "poiId": "1000559885",
                "rpFlag": "16",
                "destSearchFlag": "03"
            },
            "predictionType": "departure",
            "predictionTime": f"{yymmdd}T{hour}:{min}:00+0900",
            "searchOption": "00",
            "tollgateCarType": "car",
            "trafficInfo": "N"
        }}
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "appKey": "your-appkey"
    }
    response = requests.post(url, json=payload, headers=headers)
    response = json.loads(response.text)
    if not('features' in response.keys()):
        return {}
    duration, distance, routes = tmap_direction_result_process(response)
    return {'duration':duration, 'distance':distance, 'routes':routes}

In [27]:
# time (초), distance (미터)
O = [36.352799521652, 127.423309425352] # 한남대학교d
D = [36.3512760051536, 127.437518299333] # 대전복합터미널
yymmdd = '2018-01-02'
hour = '02'
min = '14'

result = tmap_past_direction(O, D, yymmdd, hour, min)
print(result)

{'duration': 382, 'distance': 2881, 'routes': [[127.4233312000267, 36.352938747730384], [127.42314788165088, 36.352991516209734], [127.42273680518322, 36.353069277779205], [127.42189243073021, 36.35327201734197], [127.42172022269875, 36.353316453639195], [127.42155912478502, 36.35336089013394], [127.42155912478502, 36.35336089013394], [127.42156467946656, 36.35337477753227], [127.42157023384627, 36.35339977477025], [127.4215674556377, 36.353424771860105], [127.42155912237028, 36.3534497688512], [127.42153967913612, 36.35346921072495], [127.42151745837255, 36.35348865254934], [127.42149246030591, 36.35349976194459], [127.42146468486081, 36.35350531637065], [127.42143413203719, 36.353505315827505], [127.42143413203719, 36.353505315827505], [127.42143412675493, 36.35369973802145], [127.42142023224082, 36.3539524866266], [127.42138132434083, 36.35478016898863], [127.42136465184417, 36.35504958230347], [127.4213590921819, 36.35521900725907], [127.42135909150271, 36.35524400439824], [127.421

---